# Projeto 2 - Como pegar dados de um site com Python? 

Pegando dados de ETFs do mundo inteiro.


### Desafio:

* Construir um código que vá no site etf.com e busque dados de todos os etfs do mercado americano e, consequentemente, do mundo. Rentabilidades, patrimônio, gestora, taxa...   
* Lembrar de sempre trazer outros tipos de cenários onde a pessoa precisa pegar dados de sites etc.


### Passo a passo:

   **Passo 1** - Definir um navegador que você irá utilizar para navegar com o Python.

   **Passo 2** - Importar os módulos e bibliotecas.
   
   **Passo 3** - Entender como funcionam requisições na internet.
   
   **Passo 4** - Conhecer e mapear o processo de coleta de dados no site do ETF.com. 
   
   **Passo 5** - Achar todos os elementos necessários dentro do HTML do site.
   
   **Passo 6** - Ler a tabela de dados.
   
   **Passo 7** - Construir a tabela final.

In [1]:
# !pip install webdriver-manager

In [2]:
# !pip install selenium

In [3]:
# !pip install html5lib

# Passo 1 - Selecionando o navegador e importando as bibliotecas

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

### beautifulsoup é outra opção par web scraping, porém ele tem menos recursos do que o Selenium. 
### beautifulsoup é mais simples e leve. Selenium demora um pouco a mais.

# Passo 2 - Chamando o navegador e acessar o site desejado

In [5]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

url = "https://www.etf.com/etfanalytics/etf-finder"

driver.get(url)

# Processo de dados

* Abrir o site - Completo
* Mudar a visualização pra 100
* Ler a tabela
* Avançar todas as páginas
* Trocar pra outra categoria
* Ler todas as tabelas dessa outra categoria

# Passo 5.1: Achar todos os elementos necessários dentro do HTML do site - Expandindo a tabela para 100 itens.

In [6]:
# Add uma pausa para carregar o javascript
time.sleep(5)

#Add o comando para mudar para o maximo de itens na visualização
#Copiar a linha com full XPatch para evitar erros
botao_100 = driver.find_element("xpath", '''html/body/div[5]/section/div/div[3]/section/div
/div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span''')

driver.execute_script("arguments[0].click();", botao_100)

# Passo 5.2: Achar todos os elementos necessários dentro do HTML do site - Pegando o número de páginas da tabela.

In [7]:
#localizando o numero de subpaginas da tabela, após a mudança da visualização

numero_paginas = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/
section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/label[2]''')

numero_paginas = numero_paginas.text.replace("of ", "")

numero_paginas = int(numero_paginas)

print(numero_paginas)

31


# Passo 6.1: Lendo a tabela de dados - Tabela 1: Fund Basics.

In [8]:
#add uma lista vazia
lista_de_tabela_por_pagina = []

#criando um loop para ler todas as paginas, copiar todas as tabelas, armazenar e depois concatenar em uma unica tabela
for pagina in range(0, numero_paginas):
    
    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/
                                    div/div/div/div[2]/section[2]/div[2]/div/table''')

    html_tabela = tabela.get_attribute("outerHTML")

    tabela_final = pd.read_html(html_tabela)[0]
    
    lista_de_tabela_por_pagina.append(tabela_final)
    
    botao_avancar_pagina = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]
                                        /section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/span[2]''')
    
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    

base_de_dados_completa = pd.concat(lista_de_tabela_por_pagina)

display(base_de_dados_completa)

,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$384.58B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$313.53B
2,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$287.40B
3,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.32B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$164.12B
...,...,...,...,...,...,...
85,AWYX,ETFMG 2x Daily Travel Tech ETF,Leveraged Equity: Global Internet & Direct Mar...,ETFMG,0.95%,$341.97K
86,TADS,The Active Dividend Stock ETF,Equity: U.S. - Total Market,"Tuttle Tactical Management, LLC",1.68%,$294.89K
87,CRYP,AdvisorShares Managed Bitcoin Strategy ETF,Asset Allocation: Global Target Outcome,AdvisorShares,1.59%,$217.68K
88,FLRU,Franklin FTSE Russia ETF,Equity: Russia - Total Market,Franklin Templeton,0.19%,$8.00K


# Passo 6.2: Ler a tabela de dados - Tabela 2: Perfomance

In [9]:
#primeira coisa é mudar a aba

botao_aba = driver.find_element("xpath", ''' /html/body/div[5]/section/div/div[3]/section/div/div/div/div/
div[2]/section[2]/div[2]/ul/li[2]/span''')

driver.execute_script("arguments[0].click();", botao_aba)

#voltar para a 1ª página

for pagina in range(0, numero_paginas):
    
    botao_avancar_pagina = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/
    div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/span[1]''')
    
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)

In [10]:
lista_de_tabela_por_pagina = []

for pagina in range(0, numero_paginas):
    
    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/
                                    div/div/div/div[2]/section[2]/div[2]/div/table''')

    html_tabela = tabela.get_attribute("outerHTML")

    tabela_final = pd.read_html(html_tabela)[0]
    
    lista_de_tabela_por_pagina.append(tabela_final)
    
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    

base_de_dados_performance = pd.concat(lista_de_tabela_por_pagina)



In [11]:
display(base_de_dados_performance)

,Ticker,Name,1 Month,3 Month,YTD,1 Year,3 Years,5 Years,10 Years,As Of Date
0,SPY,SPDR S&P 500 ETF Trust,4.03%,8.84%,3.52%,-9.95%,7.75%,8.99%,12.39%,01/20/23
1,IVV,iShares Core S&P 500 ETF,4.06%,8.84%,3.51%,-9.95%,7.75%,9.03%,12.44%,01/20/23
2,VTI,Vanguard Total Stock Market ETF,4.49%,8.98%,3.96%,-10.36%,7.24%,8.49%,12.02%,01/20/23
3,VOO,Vanguard S&P 500 ETF,4.02%,8.82%,3.52%,-9.98%,7.73%,9.02%,12.44%,01/20/23
4,QQQ,Invesco QQQ Trust,4.88%,5.30%,6.16%,-21.28%,8.84%,11.98%,16.55%,01/20/23
...,...,...,...,...,...,...,...,...,...,...
85,AWYX,ETFMG 2x Daily Travel Tech ETF,36.11%,38.66%,31.27%,-48.46%,--,--,--,01/20/23
86,TADS,The Active Dividend Stock ETF,0%,0%,0%,0%,--,--,--,01/20/23
87,CRYP,AdvisorShares Managed Bitcoin Strategy ETF,--,--,--,--,--,--,--,01/20/23
88,FLRU,Franklin FTSE Russia ETF,0%,0%,0%,-66.37%,-31.18%,--,--,01/20/23


In [12]:
#fechar o navegador
driver.quit()

# Passo 7: Construindo a tabela final

In [13]:
#mudar o index para a coluna Ticker
base_de_dados_completa = base_de_dados_completa.set_index("Ticker")

display(base_de_dados_completa)

,Name,Segment,Issuer,Expense Ratio,AUM
Ticker,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$384.58B
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$313.53B
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$287.40B
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.32B
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$164.12B
...,...,...,...,...,...
AWYX,ETFMG 2x Daily Travel Tech ETF,Leveraged Equity: Global Internet & Direct Mar...,ETFMG,0.95%,$341.97K
TADS,The Active Dividend Stock ETF,Equity: U.S. - Total Market,"Tuttle Tactical Management, LLC",1.68%,$294.89K
CRYP,AdvisorShares Managed Bitcoin Strategy ETF,Asset Allocation: Global Target Outcome,AdvisorShares,1.59%,$217.68K


In [14]:
#mudando o index e selecionando somente as colunas desejas da tabela perfomance
base_de_dados_performance = base_de_dados_performance.set_index("Ticker")
base_de_dados_performance = base_de_dados_performance[['1 Year', '5 Years', '10 Years']]

display(base_de_dados_performance)

,1 Year,5 Years,10 Years
Ticker,,,
SPY,-9.95%,8.99%,12.39%
IVV,-9.95%,9.03%,12.44%
VTI,-10.36%,8.49%,12.02%
VOO,-9.98%,9.02%,12.44%
QQQ,-21.28%,11.98%,16.55%
...,...,...,...
AWYX,-48.46%,--,--
TADS,0%,--,--
CRYP,--,--,--


In [15]:
#Unindo as tabelas 
base_de_dados_final = base_de_dados_completa.join(base_de_dados_performance)
display(base_de_dados_final)

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,5 Years,10 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$384.58B,-9.95%,8.99%,12.39%
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$313.53B,-9.95%,9.03%,12.44%
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$287.40B,-10.36%,8.49%,12.02%
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.32B,-9.98%,9.02%,12.44%
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$164.12B,-21.28%,11.98%,16.55%
...,...,...,...,...,...,...,...,...
AWYX,ETFMG 2x Daily Travel Tech ETF,Leveraged Equity: Global Internet & Direct Mar...,ETFMG,0.95%,$341.97K,-48.46%,--,--
TADS,The Active Dividend Stock ETF,Equity: U.S. - Total Market,"Tuttle Tactical Management, LLC",1.68%,$294.89K,0%,--,--
CRYP,AdvisorShares Managed Bitcoin Strategy ETF,Asset Allocation: Global Target Outcome,AdvisorShares,1.59%,$217.68K,--,--,--


# Analisar os dados de rentabilidade?

### Desafio:

* Construir um código que faça um ranking dos melhores fundos em três períodos de rentabilidade diferentes. Iremos selecionar os 10 melhores posicionados no quesito rentabilidade no curto e no longo prazo.


### Passo a passo:
   
   **Passo 1** - Escolher quais períodos de rentabilidade iremos analisar.
   
   **Passo 2** - Retirar os dados faltantes. Fundos que não possuem os períodos necessários, ficarão de fora. 

   **Passo 3** - Transformar as rentabilidades em números decimais, ao invés de string. 
   
   **Passo 4** - Juntar as tabelas. 
   
   **Passo 5** - Filtrar os ETFs alavancados. 
   
   **Passo 6** - Fazer os rankings de rentabilidade por período e soma-los. 
   
   **Passo 7** - Ordenar do menor ranking pro maior, encontrando os "melhores" ETFs.

# Passo 1: Escolher quais períodos de rentabilidade iremos analisar.

Janelas de 1, 5 e 10 anos. 

In [23]:
base_de_dados_performance = base_de_dados_performance[['1 Year', '5 Years', '10 Years']]

display(base_de_dados_performance)

,1 Year,5 Years,10 Years
Ticker,,,
SPY,-9.95%,8.99%,12.39%
IVV,-9.95%,9.03%,12.44%
VTI,-10.36%,8.49%,12.02%
VOO,-9.98%,9.02%,12.44%
QQQ,-21.28%,11.98%,16.55%
...,...,...,...
SZK,36.84%,-20.50%,-22.95%
LD,-8.85%,-5.95%,-1.99%
ERUS,-78.73%,-23.10%,-13.14%


# Passo 2: Retirar os dados faltantes. 
Fundos que não possuem os períodos necessários, ficarão de fora. 

In [24]:
#substituir a informação faltante da tabela por NaN e dropar os mesmos
base_de_dados_performance = base_de_dados_performance.replace("--", pd.NA)

base_de_dados_performance = base_de_dados_performance.dropna()

display(base_de_dados_performance)

,1 Year,5 Years,10 Years
Ticker,,,
SPY,-9.95%,8.99%,12.39%
IVV,-9.95%,9.03%,12.44%
VTI,-10.36%,8.49%,12.02%
VOO,-9.98%,9.02%,12.44%
QQQ,-21.28%,11.98%,16.55%
...,...,...,...
SZK,36.84%,-20.50%,-22.95%
LD,-8.85%,-5.95%,-1.99%
ERUS,-78.73%,-23.10%,-13.14%


# Passo 4: Transformar as rentabilidades em números decimais

In [25]:
#Remover o simbolo % e depois dividir por 100 para ter a porcentagem do dado
base_de_dados_performance['1 Year'] = (base_de_dados_performance['1 Year'].
                                        str.rstrip('%').astype(float)/100)

base_de_dados_performance['5 Years'] = (base_de_dados_performance['5 Years'].
                                        str.rstrip('%').astype(float)/100)

base_de_dados_performance['10 Years'] = (base_de_dados_performance['10 Years'].
                                        str.rstrip('%').astype(float)/100)

display(base_de_dados_performance)

,1 Year,5 Years,10 Years
Ticker,,,
SPY,-0.0995,0.0899,0.1239
IVV,-0.0995,0.0903,0.1244
VTI,-0.1036,0.0849,0.1202
VOO,-0.0998,0.0902,0.1244
QQQ,-0.2128,0.1198,0.1655
...,...,...,...
SZK,0.3684,-0.2050,-0.2295
LD,-0.0885,-0.0595,-0.0199
ERUS,-0.7873,-0.2310,-0.1314


In [26]:
base_de_dados_final = base_de_dados_completa.join(base_de_dados_performance, how = "inner")
#how = inner serve para manter na tabela apenas os dados que estiverem nas duas, do contrario, aquelas dropadas continuaram nesta tabela

display(base_de_dados_final)

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,5 Years,10 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$384.58B,-0.0995,0.0899,0.1239
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$313.53B,-0.0995,0.0903,0.1244
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$287.40B,-0.1036,0.0849,0.1202
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.32B,-0.0998,0.0902,0.1244
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$164.12B,-0.2128,0.1198,0.1655
...,...,...,...,...,...,...,...,...
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.28M,0.3684,-0.2050,-0.2295
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.23M,-0.0885,-0.0595,-0.0199
ERUS,iShares MSCI Russia ETF,Equity: Russia - Total Market,Blackrock,0.08%,$961.13K,-0.7873,-0.2310,-0.1314


# Passo 5: Remover os ETFs alavancados.

In [27]:
base_de_dados_final = base_de_dados_final[~base_de_dados_final['Segment'].str.contains("Leveraged")]
#o ~ antes do comando, serve para fazer o contrario
display(base_de_dados_final)

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,5 Years,10 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$384.58B,-0.0995,0.0899,0.1239
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$313.53B,-0.0995,0.0903,0.1244
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$287.40B,-0.1036,0.0849,0.1202
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.32B,-0.0998,0.0902,0.1244
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$164.12B,-0.2128,0.1198,0.1655
...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.92M,-0.1240,-0.2535,-0.2925
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.28M,0.3684,-0.2050,-0.2295
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.23M,-0.0885,-0.0595,-0.0199


# Passo 6: Fazer os rankings de rentabilidade por período e soma-los. 

In [28]:
base_de_dados_final['rank_1_ano'] = base_de_dados_final['1 Year'].rank(ascending = False)
base_de_dados_final['rank_5_anos'] = base_de_dados_final['5 Years'].rank(ascending = False)
base_de_dados_final['rank_10_anos'] = base_de_dados_final['10 Years'].rank(ascending = False)

display(base_de_dados_final)

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,5 Years,10 Years,rank_1_ano,rank_5_anos,rank_10_anos
Ticker,,,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$384.58B,-0.0995,0.0899,0.1239,577.5,108.0,78.5
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$313.53B,-0.0995,0.0903,0.1244,577.5,101.5,76.0
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$287.40B,-0.1036,0.0849,0.1202,593.0,137.0,104.5
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.32B,-0.0998,0.0902,0.1244,581.0,104.0,76.0
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$164.12B,-0.2128,0.1198,0.1655,793.0,22.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.92M,-0.1240,-0.2535,-0.2925,655.5,811.0,817.0
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.28M,0.3684,-0.2050,-0.2295,22.0,801.0,806.0
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.23M,-0.0885,-0.0595,-0.0199,542.0,760.0,703.5


In [29]:
base_de_dados_final['rank_final'] =  (base_de_dados_final['rank_1_ano'] + 
                                      base_de_dados_final['rank_5_anos'] + 
                                      base_de_dados_final['rank_10_anos'])

display(base_de_dados_final)

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,5 Years,10 Years,rank_1_ano,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$384.58B,-0.0995,0.0899,0.1239,577.5,108.0,78.5,764.0
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$313.53B,-0.0995,0.0903,0.1244,577.5,101.5,76.0,755.0
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$287.40B,-0.1036,0.0849,0.1202,593.0,137.0,104.5,834.5
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.32B,-0.0998,0.0902,0.1244,581.0,104.0,76.0,761.0
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$164.12B,-0.2128,0.1198,0.1655,793.0,22.0,13.0,828.0
...,...,...,...,...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.92M,-0.1240,-0.2535,-0.2925,655.5,811.0,817.0,2283.5
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.28M,0.3684,-0.2050,-0.2295,22.0,801.0,806.0,1629.0
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.23M,-0.0885,-0.0595,-0.0199,542.0,760.0,703.5,2005.5


# Passo 8: Gran finale - Ordenar do menor ranking pro maior, encontrando os "melhores" ETFs.

In [30]:
melhores_etfs = base_de_dados_final.sort_values(by = "rank_final")

display(melhores_etfs)

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,5 Years,10 Years,rank_1_ano,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
TAN,Invesco Solar ETF,Equity: Global Renewable Energy,Invesco,0.69%,$2.53B,0.1863,0.2532,0.1735,42.0,1.0,8.0,51.0
ICLN,iShares Global Clean Energy ETF,Equity: Global Renewable Energy,Blackrock,0.40%,$5.09B,0.0949,0.1798,0.1281,78.5,4.0,62.0,144.5
KBWP,Invesco KBW Property & Casualty Insurance ETF,Equity: U.S. Property & Casualty Insurance,Invesco,0.35%,$387.61M,0.0967,0.1017,0.1342,75.0,49.0,45.0,169.0
XLV,Health Care Select Sector SPDR Fund,Equity: U.S. Health Care,State Street Global Advisors,0.10%,$40.50B,0.0397,0.1053,0.1409,123.0,39.0,31.5,193.5
IYH,iShares U.S. Healthcare ETF,Equity: U.S. Health Care,Blackrock,0.39%,$3.28B,0.0295,0.1008,0.1379,139.5,51.5,36.0,227.0
...,...,...,...,...,...,...,...,...,...,...,...,...
SMDD,ProShares UltraPro Short MidCap400,Inverse Equity: U.S. - Mid Cap,ProShares,0.95%,$6.49M,-0.2593,-0.4207,-0.4131,814.0,829.0,827.0,2470.0
YANG,Direxion Daily FTSE China Bear 3X Shares,Inverse Equity: China - Large Cap,Direxion,1.00%,$187.86M,-0.5382,-0.2840,-0.3934,831.0,817.0,824.0,2472.0
DUG,ProShares UltraShort Oil & Gas,"Inverse Equity: U.S. Oil, Gas & Consumable Fuels",ProShares,0.95%,$41.06M,-0.6700,-0.4115,-0.2891,833.0,827.0,816.0,2476.0


In [31]:
melhores_etfs.head(10)

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,5 Years,10 Years,rank_1_ano,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
TAN,Invesco Solar ETF,Equity: Global Renewable Energy,Invesco,0.69%,$2.53B,0.1863,0.2532,0.1735,42.0,1.0,8.0,51.0
ICLN,iShares Global Clean Energy ETF,Equity: Global Renewable Energy,Blackrock,0.40%,$5.09B,0.0949,0.1798,0.1281,78.5,4.0,62.0,144.5
KBWP,Invesco KBW Property & Casualty Insurance ETF,Equity: U.S. Property & Casualty Insurance,Invesco,0.35%,$387.61M,0.0967,0.1017,0.1342,75.0,49.0,45.0,169.0
XLV,Health Care Select Sector SPDR Fund,Equity: U.S. Health Care,State Street Global Advisors,0.10%,$40.50B,0.0397,0.1053,0.1409,123.0,39.0,31.5,193.5
IYH,iShares U.S. Healthcare ETF,Equity: U.S. Health Care,Blackrock,0.39%,$3.28B,0.0295,0.1008,0.1379,139.5,51.5,36.0,227.0
VHT,Vanguard Health Care ETF,Equity: U.S. Health Care,Vanguard,0.10%,$17.17B,0.0244,0.1002,0.1402,148.0,54.5,33.5,236.0
RWJ,Invesco S&P SmallCap 600 Revenue ETF,Equity: U.S. - Small Cap,Invesco,0.39%,$1.21B,0.0105,0.1108,0.1260,178.0,30.0,69.0,277.0
FXZ,First Trust Materials AlphaDEX Fund,Equity: U.S. Materials,First Trust,0.61%,$1.78B,0.1325,0.0975,0.1112,59.0,66.0,155.5,280.5
SCHD,Schwab U.S. Dividend Equity ETF,Equity: U.S. - High Dividend Yield,Charles Schwab,0.06%,$47.62B,-0.0010,0.1085,0.1327,208.0,32.0,51.0,291.0
